<br/>

<img src="https://uploads-ssl.webflow.com/614b1fe22fa8b90ef41aeffe/6265cb48f9496b1cefc9ab75_logotipo-mbit-39.png" width="200px" align="right" CLASS="TextWrap" style="background-color:#2a3f3f;">

<h1><font color="#2a3f3f" size=5>Cabifly APP</font></h1>

<br/>
<div style="text-align: right">
<font color="#2a3f3f" size=3>Adrian Sanchez - adrian.sanchez@mbitschool.com</font><br>
<font color="#2a3f3f" size=3>Máster en Data Engineering </font><br>
</div>


¡La App principal de Cabifly!

Esta libreta simula la aplicación de cabifly utilizando la librería `ipyleaftlet` una versión interactiva de Folium.

En esta libreta:

- Utilizamos la API de drones para obtener su ubicación y pintarlos de manera interactiva en el mapa
- Utilizamos la API de trips para solicitar un nuevo viaje y determinar la ubicación del drone más idóneo

__TAREAS:__

- No tienes que hacer nada en esta libreta, solo comprobar que funciona con el resto de componentes que estás construyendo!
- Acuérdate de actualizar las URLs cuando quieras utilizar tus microservicios en AWS

## Configuración de la interfaz

Utilizamos el paquete ipyleaflet para que el mapa sea interactivo

In [ ]:
from ipyleaflet import Map, Marker, Icon

import requests
import time

In [ ]:
geo_madrid = (40.4168, -3.7038)

In [ ]:
m = Map(
    center=geo_madrid, 
    zoom=15
)

Esta función convierte los datos de los drones en marcadores del mapa

In [ ]:
def get_marker(drone):
    return Marker(
        icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/1830/1830867.png', icon_size=[40, 40], icon_anchor=[40,40]),
        location=(drone["location"]["coordinates"][1], drone["location"]["coordinates"][0]),
        title=drone["drone_id"],
        draggable=False
    )

### Carga de datos inicial

Cargamos los drones de la API y los añadimos al mapa

In [ ]:
# Get drones from mongodb
drones = requests.get("http://localhost:8000/drones").json()

In [ ]:
markers = {d["drone_id"]: get_marker(d) for d in drones}

In [ ]:
for mkr in markers.values():
    m.add_layer(mkr)

### Marcador para posición del usuario

Permitirá obtener su posición para llamar a la API de trips

In [ ]:
user_location = Marker(location=geo_madrid)
m.add_layer(user_location)

In [ ]:
user_location.location

### Mostramos y actualizamos el mapa

In [ ]:
m

### Llamadas a la API para crear y consultar los viajes

In [ ]:
trip = requests.post(
    "http://localhost:8001/users/adrian/trips", 
    json={
        "lon": user_location.location[1], 
        "lat": user_location.location[0]
    }
).json()

In [ ]:
trip

In [ ]:
# Get drones from mongodb
trips = requests.get("http://localhost:8001/users/adrian/trips").json()

In [ ]:
trips

### Bucle de actualización del mapa

Lo ejecutamos en un thread para no bloquear la libreta

In [ ]:
import threading

In [ ]:
# Para pararlo cambiar esta variable
running = True

In [ ]:
def update_map():
    while running:
        # Get drones from mongodb
        # drones = requests.get()
        drones = requests.get("http://localhost:8000/drones").json()

        for d in drones:
            markers[d["drone_id"]].location = (d["location"]["coordinates"][1], d["location"]["coordinates"][0])

        time.sleep(5)

In [ ]:
thread = threading.Thread(target=update_map)

In [ ]:
thread.start()

In [ ]:
thread.is_alive()